<center>
<img src="./pictures/ICA.png" style="float:left; max-width: 80px; display: inline" alt="ICA" /></a> 
<a href="https://r3asc18.sciencesconf.org/" ><img src="./pictures/web_R3ASC_20.png" style="max-width: 500px; display: inline"  alt="R3ASC"/></a> 
<img src="./pictures/logo-insa.jpg" style="float:right; max-width: 120px; display: inline" alt="INSA" /></a>
</center>

# Optimization of a motor/reducer set for a high dynamic application
*Written by Marc Budinger, INSA Toulouse, France*

For applications wigh high bandwith mission profile and with high acceleration, the choice of a motor and a gearbox depends on a compromise function of the motor inertia and the reduction ratio.  

**Scipy** and **math** packages will be used for this notebook in order to illustrate the optimization algorithms of python.

In [2]:
import scipy
import scipy.optimize
from math import pi
import timeit

## Objectives and specifications

The objective is to select the reduction ratio of a gear reducer in order to minimize the mass of the motor.

The application have to ensure :
- a max force $F_{load}$ of $73 kN$ and a max acceleration of $a_{max}=11.7 m/s²$  
- a max speed $v_{max}$ of 0.22 m/s

We will give here an example based on a linear actuator with a preselected roller screw (pitch of 10 mm/rev).
We assume here, for simplification, the efficiency equal to 100%.

*EMA components:*
![EMA](./pictures/EMA_VEGA.png)


In [83]:
# Specifications
Max_speed=.22     # [m/s] max speed
Max_acceleration=11.7   # [m/s²] max acceleration (comined with max force)
Max_load=73000    # [N] max force
 
# Assumptions
Pitch=10e-3/2/pi  # [m/rad] roller screw pitch


## Equations

#### Sizing scenarios 

The brushless electric motor will be sized considering the maximum transient torque it has to deliver $T_{max}$. This torque is calculated as the sum of maximal transient application load demand $T_{load}$ and the additional inertia load, product of the motor inertia $J_{mot}$ and its acceleration rate:

$T_{max}=T_{load}+J_{mot}.\left(\frac{d\Omega}{dt}\right)_{max}$

where:
- $T_{load}=F_{load}.p$ 
- $\left(\frac{d\Omega}{dt}\right)_{max}=p.a_{max}$  
with $p$ the pitch in $[m/rad]$

The maximal speed of the motor have also to be compared to the maximal speed requested by the application:  
$\Omega_{max,motor}>p.v_{max}$

#### Parameter estimation with scaling laws

The needed characterics of the motor can be estinated through the following scaling laws:

$M_{mot}=M_{ref}.\left(\frac{T_{max}}{T_{max,ref}}\right)^{3/3.5}$  
$J_{mot}=J_{ref}.\left(\frac{T_{max}}{T_{max,ref}}\right)^{5/3.5}$  
$\Omega_{max,motor}=\Omega_{ref}.\left(\frac{T_{max}}{T_{max,ref}}\right)^{-1/3.5}$  

where the reference values are: $T_{max,ref}=13.4 N.m$, $\Omega_{ref}=754 rad/s$, $J_{ref}=2.9.10^{-4} kg.m^2$, $M_{ref}=3.8 kg$.


## Sizing code (exercice)

Set up a sizing code to select the reduction ratio that minimizes the motor mass. Pay attention to the algebraic loops present in this design problem. The sizing code should take the form of a function: 
- taken as first parameter: the optimization variables (in an array)
- given an evaluation of the objective or the the constraints (depending of the arg parameter)


In [84]:
# Reference parameters for scaling laws
Tmax_ref=13.4    # [N.m]
Wmax_ref=754    # [rad/s]
Jref=2.9e-4/4     # [kg.m²]
Mref=3.8        # [kg]

# -----------------------
# sizing code
# -----------------------
# inputs: 
# - param: optimisation variables vector (reduction ratio, oversizing coefficient)
# - arg: selection of output  
# output: 
# - objective if arg='Obj', problem characteristics if arg='Prt', constraints other else

def SizingCode(param, arg):
# Student part -----------------------------

# Variables

# Equations



# Student part -----------------------------


# Objective and contraints
    if arg=='Obj':
# Student part ----------------------------
        return #####
    elif arg=='Prt':
        print("* Optimisation variables:")
# Student part -----------------------------
        print("* Components characteristics:")
# Student part -----------------------------
        print("* Constraints (should be >0):")
# Student part -----------------------------
    else:
# Student part ----------------------------
        return [######, ######]


## Optimization problem


We will now use the [opmization algorithms](https://docs.scipy.org/doc/scipy/reference/optimize.html) of the Scipy package to solve and optimize the configuration. We use here the SLQP algorithm without explicit expression of the gradient (Jacobian). A short course on Multidisplinary Gradient optimization algorithms and gradient optimization algorithm is given [here](http://mdolab.engin.umich.edu/sites/default/files/Martins-MDO-course-notes.pdf):
> Joaquim R. R. A. Martins (2012). A Short Course on Multidisciplinary Design Optimization. Univeristy of Michigan


The first step is to give an initial value of optimisation variables:

In [85]:
#Variables d'optimisation
N_reduc=2
k_oversizing=3

# Vector of parameters
parameters = scipy.array((N_reduc, k_oversizing))


We can print of the characterisitcs of the problem before optimization with the intitial vector of optimization variables:

In [86]:
# Initial characteristics before optimization 
print("-----------------------------------------------")
print("Initial characteristics before optimization :")

SizingCode(parameters, 'Prt')
print("-----------------------------------------------")


-----------------------------------------------
Initial characteristics before optimization :
* Optimisation variables:
           Reduction ratio N_reduc = 2.00
           Oversizing coefficient k_oversizing = 3.00
* Components characteristics:
           Motor torque = 174.27 N.m
           Motor mass = 34.26 kg
* Constraints (should be >0):
           Speed margin Speed_motor-N_reduc*Max_speed/Pitch= 85.823
           Torque margin Tem_est-Tem_max= 74.559 
-----------------------------------------------


Then we can solve the problem and print of the optimized solution:

In [87]:
# optimization with SLSQP algorithm
contrainte=lambda x: SizingCode(x, 'Const')
objectif=lambda x: SizingCode(x, 'Obj')
result = scipy.optimize.fmin_slsqp(func=objectif, x0=parameters, 
                                   bounds=[(.1,10),(1,10)],
                                   f_ieqcons=contrainte, iter=100, acc=1e-8)

# Final characteristics after optimization 
print("-----------------------------------------------")
print("Final characteristics after optimization :")

SizingCode(result, 'Prt')
print("-----------------------------------------------")



Optimization terminated successfully.    (Exit mode 0)
            Current function value: 0.9393256142050361
            Iterations: 7
            Function evaluations: 28
            Gradient evaluations: 7
-----------------------------------------------
Final characteristics after optimization :
* Optimisation variables:
           Reduction ratio N_reduc = 4.03
           Oversizing coefficient k_oversizing = 1.34
* Components characteristics:
           Motor torque = 38.52 N.m
           Motor mass = 9.39 kg
* Constraints (should be >0):
           Speed margin Speed_motor-N_reduc*Max_speed/Pitch= 0.000
           Torque margin Tem_est-Tem_max= 0.000 
-----------------------------------------------
